In [1]:
from joblib import Parallel, delayed
import numpy as np
import copy
import numpy.ma as ma
import csv
from statistics import median, mean
from numpy import genfromtxt

import nimfa  # non-negative matrix factorization
import random

#polo
from polo import optimal_leaf_ordering
from scipy.spatial.distance import pdist
from fastcluster import linkage
from scipy.cluster.hierarchy import leaves_list

import sys
sys.path.append("..")
import utils as ut
import distance_correlation as dc
import STMF as tmf  # sparse tropical matrix factorization

/home/amra/miniconda3/envs/NMF_vs_TMF/lib/python3.6/site-packages/nimfa/examples/cbcl_images.py:98: UserWarning: PIL must be installed to run CBCL images example.
  warn("PIL must be installed to run CBCL images example.")
/home/amra/miniconda3/envs/NMF_vs_TMF/lib/python3.6/site-packages/nimfa/examples/orl_images.py:110: UserWarning: PIL must be installed to run ORL images example.
  warn("PIL must be installed to run ORL images example.")


In [2]:
def run_experiment(X_maxplus_orig, X_missing_values, X_maxplus, rank_param, max_iter, missing_value, init, init_tmf, repeat):
    # NMF
    #print("NMF")
    errors_nmf_maxplus, corr_nmf_maxplus = np.array([]), np.array([])
    approx_nmf_maxplus = []
    error_iter, corr_iter = np.array([]), np.array([])
    correlations, approximations = [], []
    model = nimfa.Nmf(X_maxplus, rank=rank_param, max_iter=max_iter, seed=init)
    nmf = model()
    approx = np.dot(nmf.basis(), nmf.coef(idx=None))
    error_iter = np.append(error_iter, ut.mean_relative_error(X_maxplus_orig, X_missing_values, approx,
                                                                      missing_value))
    correlation, a, b = dc.dcor(X_maxplus_orig, approx)
    corr_iter = np.append(corr_iter, correlation)
    correlations.append(correlation)
    approximations.append(approx)
    #print("rank " + str(rank_param) + ": " + str(corr_iter[0]))
    errors_nmf_maxplus, corr_nmf_maxplus = np.append(errors_nmf_maxplus, error_iter), \
                                               np.append(corr_nmf_maxplus, corr_iter)
    # STMF
    #print("STMF")
    errors_trop_maxplus, corr_trop_maxplus = np.array([]), np.array([])
    approx_trop_maxplus = []
    factor_U, factor_V = [], []
    
    for i in range(rank_param, rank_param+1):
        error_iter, corr_iter = np.array([]), np.array([])
        correlations, approximations = [], []
        for j in range(0, repeat):
            model = tmf.STMF(rank=i, criterion='iterations', max_iter=max_iter, initialization=init_tmf)
            model.fit(X_maxplus)
            approx = model.predict_all()
            error_iter = np.append(error_iter, ut.mean_relative_error(X_maxplus_orig, X_missing_values, approx,
                                                                      missing_value))
            correlation, a, b = dc.dcor(X_maxplus_orig, approx)
            corr_iter = np.append(corr_iter, correlation)
            correlations.append(correlation)
            approximations.append(approx)
        #print("rank " + str(i) + ": mean is " + str(mean(correlations)) + ", median is " + str(median(correlations)) + ", max is " + str(max(correlations)))
        errors_trop_maxplus, corr_trop_maxplus =  np.append(errors_trop_maxplus, error_iter), \
                                                  np.append(corr_trop_maxplus, corr_iter)
    
    return errors_trop_maxplus, corr_trop_maxplus

In [3]:
def return_permData(data_param):
    data = copy.deepcopy(data_param)
    D = pdist(data, 'euclidean')  # distance
    Z = linkage(D, 'ward')
    optimal_Z = optimal_leaf_ordering(Z, D)
    opt_order = leaves_list(optimal_Z)
    data = data[opt_order]
   
    data = data.T  # transpose
    D = pdist(data, 'euclidean')  # distance
    Z = linkage(D, 'ward')
    optimal_Z = optimal_leaf_ordering(Z, D)
    opt_order_columns = leaves_list(optimal_Z)
    data = data[opt_order_columns]
    
    return data.T, opt_order, opt_order_columns

In [4]:
def generate_wordcloud(seed):
    np.random.seed(seed)
    random.seed(seed)
    m = 500  # number of rows, 500
    n = 300   # number of columns, 300
    rank = 3  # rank 3
    missing_value = 0  # 99 is better than 0, because 0 can be the real value in data; 0 because of nmf!
    repeat = 100  # repeat 100
    max_iter = 500
    sparsity = 0.2  # 20%
    init = 'nndsvd'  # random, random_c, random_vcol, nndsvd
    init_tmf = 'random_vcol'

    # basic and missing matrices
    X_temp = genfromtxt("Synthetic_matrix_1.csv", delimiter=",")
    n_rows, n_columns =  X_temp.shape[0], X_temp.shape[1]
    X_temp = ma.masked_array(X_temp, mask=np.zeros((n_rows, n_columns)))

    row_perm = np.random.permutation(n_rows)
    columns_perm = np.random.permutation(n_columns)

    X_temp = X_temp[:, columns_perm]
    X_temp = X_temp[row_perm] 

    ut.check_zeros(X_temp)
    X_basic = copy.deepcopy(X_temp) # original matrix 
    X_missing = ut.create_matrix_with_missing_values(X_temp, sparsity, missing_value)
    
    # spectral
    X_maxplus_orig = copy.deepcopy(X_basic)  # original matrix
    X_missing_values = copy.deepcopy(X_missing)

    X_maxplus, row_perm, col_perm = ut.cluster_matrix(X_missing_values)  # spectral biclustering
    X_maxplus = ma.masked_equal(X_maxplus, missing_value)  # create masked array
    X_maxplus /= X_maxplus.sum()

    X_maxplus_orig = X_maxplus_orig[row_perm]
    X_maxplus_orig = X_maxplus_orig[:, col_perm]
    X_missing_values = X_missing_values[row_perm]
    X_missing_values = X_missing_values[:, col_perm]
    
    err_spectral, corr_spectral = run_experiment(X_maxplus_orig, X_missing_values, X_maxplus, rank, max_iter, missing_value, init, init_tmf, repeat)
    
    # polo
    X_maxplus_orig = copy.deepcopy(X_basic)  # original matrix
    X_missing_values = copy.deepcopy(X_missing)

    X_maxplus, row_perm, col_perm = return_permData(X_missing_values)  # polo ordering
    X_maxplus = ma.masked_equal(X_maxplus, missing_value)  # create masked array
    X_maxplus /= X_maxplus.sum()

    X_maxplus_orig = X_maxplus_orig[row_perm]
    X_maxplus_orig = X_maxplus_orig[:, col_perm]
    X_missing_values = X_missing_values[row_perm]
    X_missing_values = X_missing_values[:, col_perm]
    
    err_polo, corr_polo = run_experiment(X_maxplus_orig, X_missing_values, X_maxplus, rank, max_iter, missing_value, init, init_tmf, repeat)
    
    # polo flipped
    X_maxplus_orig = copy.deepcopy(X_basic)  # original matrix
    X_missing_values = copy.deepcopy(X_missing)

    X, row_perm, col_perm = return_permData(X_missing_values)  # polo ordering
    row_perm = row_perm[::-1] # flipped 
    col_perm = col_perm[::-1]
    X_maxplus = X_missing_values[row_perm]
    X_maxplus = X_maxplus[:, col_perm]
    X_maxplus = ma.masked_equal(X_maxplus, missing_value)  # create masked array
    X_maxplus /= X_maxplus.sum()

    X_maxplus_orig = X_maxplus_orig[row_perm]
    X_maxplus_orig = X_maxplus_orig[:, col_perm]
    X_missing_values = X_missing_values[row_perm]
    X_missing_values = X_missing_values[:, col_perm]
    
    err_polo_flipped, corr_polo_flipped = run_experiment(X_maxplus_orig, X_missing_values, X_maxplus, rank, max_iter, missing_value, init, init_tmf, repeat)
    
    # polo flipped rows
    X_maxplus_orig = copy.deepcopy(X_basic)  # original matrix
    X_missing_values = copy.deepcopy(X_missing)

    X, row_perm, col_perm = return_permData(X_missing_values)  # polo ordering
    row_perm = row_perm[::-1] # flipped 
    X_maxplus = X_missing_values[row_perm]
    X_maxplus = ma.masked_equal(X_maxplus, missing_value)  # create masked array
    X_maxplus /= X_maxplus.sum()

    X_maxplus_orig = X_maxplus_orig[row_perm]
    X_missing_values = X_missing_values[row_perm]
    
    err_polo_flipped_rows, corr_polo_flipped_rows = run_experiment(X_maxplus_orig, X_missing_values, X_maxplus, rank, max_iter, missing_value, init, init_tmf, repeat)
    
    # random rows
    X_maxplus_orig = copy.deepcopy(X_basic)  # original matrix
    X_missing_values = copy.deepcopy(X_missing)

    n_rows =  X_missing_values.shape[0]
    row_perm = np.random.permutation(n_rows)
    X_maxplus = X_missing_values[row_perm]  # random permutation of rows
    X_maxplus = ma.masked_equal(X_maxplus, missing_value)  # create masked array
    X_maxplus /= X_maxplus.sum()

    X_maxplus_orig = X_maxplus_orig[row_perm]
    X_missing_values = X_missing_values[row_perm]
    
    err_random_rows, corr_random_rows = run_experiment(X_maxplus_orig, X_missing_values, X_maxplus, rank, max_iter, missing_value, init, init_tmf, repeat)
    
    # random columns
    X_maxplus_orig = copy.deepcopy(X_basic)  # original matrix
    X_missing_values = copy.deepcopy(X_missing)

    n_columns =  X_missing_values.shape[1]
    columns_perm = np.random.permutation(n_columns)
    X_maxplus = X_missing_values[:, columns_perm]  # random permutation of rows
    X_maxplus = ma.masked_equal(X_maxplus, missing_value)  # create masked array
    X_maxplus /= X_maxplus.sum()

    X_maxplus_orig = X_maxplus_orig[:, columns_perm]
    X_missing_values = X_missing_values[:, columns_perm]
    
    err_random_columns, corr_random_columns = run_experiment(X_maxplus_orig, X_missing_values, X_maxplus, rank, max_iter, missing_value, init, init_tmf, repeat)
    
    # Col_inc_max and row_inc_min
    X_maxplus_orig = copy.deepcopy(X_basic)  # original matrix
    X_missing_values = copy.deepcopy(X_missing)

    row_perm = np.argsort(np.min(X_maxplus_orig, axis = 1))
    columns_perm = np.argsort(np.max(X_maxplus_orig, axis = 0))

    X_maxplus = X_missing_values[:, columns_perm] 
    X_maxplus = X_maxplus[row_perm] 
    X_maxplus = ma.masked_equal(X_maxplus, missing_value)  # create masked array
    X_maxplus /= X_maxplus.sum()

    X_maxplus_orig = X_maxplus_orig[:, columns_perm]
    X_maxplus_orig = X_maxplus_orig[row_perm] 
    X_missing_values = X_missing_values[:, columns_perm]
    X_missing_values = X_missing_values[row_perm]
    
    err_mix_1, corr_mix_1 = run_experiment(X_maxplus_orig, X_missing_values, X_maxplus, rank, max_iter, missing_value, init, init_tmf, repeat)
    
    # Col_inc_max and row_dec_min
    X_maxplus_orig = copy.deepcopy(X_basic)  # original matrix
    X_missing_values = copy.deepcopy(X_missing)

    row_perm = np.argsort(np.min(X_maxplus_orig, axis = 1))[::-1]
    columns_perm = np.argsort(np.max(X_maxplus_orig, axis = 0))

    X_maxplus = X_missing_values[:, columns_perm] 
    X_maxplus = X_maxplus[row_perm] 
    X_maxplus = ma.masked_equal(X_maxplus, missing_value)  # create masked array
    X_maxplus /= X_maxplus.sum()

    X_maxplus_orig = X_maxplus_orig[:, columns_perm]
    X_maxplus_orig = X_maxplus_orig[row_perm] 
    X_missing_values = X_missing_values[:, columns_perm]
    X_missing_values = X_missing_values[row_perm]
    
    err_mix_2, corr_mix_2 = run_experiment(X_maxplus_orig, X_missing_values, X_maxplus, rank, max_iter, missing_value, init, init_tmf, repeat)
    
    # MEAN VALUE
    X_maxplus_orig = copy.deepcopy(X_basic)  # original matrix
    X_missing_values = copy.deepcopy(X_missing)

    row_perm = np.argsort(np.mean(X_missing_values, axis = 1))
    X_maxplus = X_maxplus_orig[row_perm]  # random permutation of rows
    X_maxplus = ma.masked_equal(X_maxplus, missing_value)  # create masked array
    X_maxplus /= X_maxplus.sum()

    X_maxplus_orig = X_maxplus_orig[row_perm]
    X_missing_values = X_missing_values[row_perm]
    
    err_mean_rows_incr, corr_mean_rows_incr = run_experiment(X_maxplus_orig, X_missing_values, X_maxplus, rank, max_iter, missing_value, init, init_tmf, repeat)
    
    X_maxplus_orig = copy.deepcopy(X_basic)  # original matrix
    X_missing_values = copy.deepcopy(X_missing)

    row_perm = np.argsort(np.mean(X_maxplus_orig, axis = 1))[::-1]
    X_maxplus = X_maxplus_orig[row_perm]  # random permutation of rows
    X_maxplus = ma.masked_equal(X_maxplus, missing_value)  # create masked array
    X_maxplus /= X_maxplus.sum()

    X_maxplus_orig = X_maxplus_orig[row_perm]
    X_missing_values = X_missing_values[row_perm]
    
    err_mean_rows_decr, corr_mean_rows_decr = run_experiment(X_maxplus_orig, X_missing_values, X_maxplus, rank, max_iter, missing_value, init, init_tmf, repeat)
    
    X_maxplus_orig = copy.deepcopy(X_basic)  # original matrix
    X_missing_values = copy.deepcopy(X_missing)

    columns_perm = np.argsort(np.mean(X_maxplus_orig, axis = 0))
    X_maxplus = X_maxplus_orig[:, columns_perm]  # random permutation of rows
    X_maxplus = ma.masked_equal(X_maxplus, missing_value)  # create masked array
    X_maxplus /= X_maxplus.sum()

    X_maxplus_orig = X_maxplus_orig[:, columns_perm]
    X_missing_values = X_missing_values[:, columns_perm]
    
    err_mean_columns_incr, corr_mean_columns_incr = run_experiment(X_maxplus_orig, X_missing_values, X_maxplus, rank, max_iter, missing_value, init, init_tmf, repeat)
    
    X_maxplus_orig = copy.deepcopy(X_basic)  # original matrix
    X_missing_values = copy.deepcopy(X_missing)

    columns_perm = np.argsort(np.mean(X_maxplus_orig, axis = 0))[::-1]
    X_maxplus = X_maxplus_orig[:, columns_perm]  # random permutation of rows
    X_maxplus = ma.masked_equal(X_maxplus, missing_value)  # create masked array
    X_maxplus /= X_maxplus.sum()

    X_maxplus_orig = X_maxplus_orig[:, columns_perm]
    X_missing_values = X_missing_values[:, columns_perm]
    
    err_mean_columns_decr, corr_mean_columns_decr = run_experiment(X_maxplus_orig, X_missing_values, X_maxplus, rank, max_iter, missing_value, init, init_tmf, repeat)
    
    # MIN VALUES
    
    X_maxplus_orig = copy.deepcopy(X_basic)  # original matrix
    X_missing_values = copy.deepcopy(X_missing)

    row_perm = np.argsort(np.min(X_maxplus_orig, axis = 1))
    X_maxplus = X_maxplus_orig[row_perm]  # random permutation of rows
    X_maxplus = ma.masked_equal(X_maxplus, missing_value)  # create masked array
    X_maxplus /= X_maxplus.sum()

    X_maxplus_orig = X_maxplus_orig[row_perm]
    X_missing_values = X_missing_values[row_perm]
    
    err, corr_min_rows_incr = run_experiment(X_maxplus_orig, X_missing_values, X_maxplus, rank, max_iter, missing_value, init, init_tmf, repeat)
    
    X_maxplus_orig = copy.deepcopy(X_basic)  # original matrix
    X_missing_values = copy.deepcopy(X_missing)

    row_perm = np.argsort(np.min(X_maxplus_orig, axis = 1))[::-1]
    X_maxplus = X_maxplus_orig[row_perm]  # random permutation of rows
    X_maxplus = ma.masked_equal(X_maxplus, missing_value)  # create masked array
    X_maxplus /= X_maxplus.sum()

    X_maxplus_orig = X_maxplus_orig[row_perm]
    X_missing_values = X_missing_values[row_perm]
    
    err, corr_min_rows_decr = run_experiment(X_maxplus_orig, X_missing_values, X_maxplus, rank, max_iter, missing_value, init, init_tmf, repeat)
    
    X_maxplus_orig = copy.deepcopy(X_basic)  # original matrix
    X_missing_values = copy.deepcopy(X_missing)

    columns_perm = np.argsort(np.min(X_maxplus_orig, axis = 0))
    X_maxplus = X_maxplus_orig[:, columns_perm]  # random permutation of rows
    X_maxplus = ma.masked_equal(X_maxplus, missing_value)  # create masked array
    X_maxplus /= X_maxplus.sum()

    X_maxplus_orig = X_maxplus_orig[:, columns_perm]
    X_missing_values = X_missing_values[:, columns_perm]
    
    err, corr_min_columns_incr = run_experiment(X_maxplus_orig, X_missing_values, X_maxplus, rank, max_iter, missing_value, init, init_tmf, repeat)
    
    X_maxplus_orig = copy.deepcopy(X_basic)  # original matrix
    X_missing_values = copy.deepcopy(X_missing)

    columns_perm = np.argsort(np.min(X_maxplus_orig, axis = 0))[::-1]
    X_maxplus = X_maxplus_orig[:, columns_perm]  # random permutation of rows
    X_maxplus = ma.masked_equal(X_maxplus, missing_value)  # create masked array
    X_maxplus /= X_maxplus.sum()

    X_maxplus_orig = X_maxplus_orig[:, columns_perm]
    X_missing_values = X_missing_values[:, columns_perm]
    
    err, corr_min_columns_decr = run_experiment(X_maxplus_orig, X_missing_values, X_maxplus, rank, max_iter, missing_value, init, init_tmf, repeat)
    
    # MAX VALUE
    X_maxplus_orig = copy.deepcopy(X_basic)  # original matrix
    X_missing_values = copy.deepcopy(X_missing)

    row_perm = np.argsort(np.max(X_maxplus_orig, axis = 1))
    X_maxplus = X_maxplus_orig[row_perm]  # random permutation of rows
    X_maxplus = ma.masked_equal(X_maxplus, missing_value)  # create masked array
    X_maxplus /= X_maxplus.sum()

    X_maxplus_orig = X_maxplus_orig[row_perm]
    X_missing_values = X_missing_values[row_perm]
    
    err, corr_max_rows_incr = run_experiment(X_maxplus_orig, X_missing_values, X_maxplus, rank, max_iter, missing_value, init, init_tmf, repeat)
    
    X_maxplus_orig = copy.deepcopy(X_basic)  # original matrix
    X_missing_values = copy.deepcopy(X_missing)

    row_perm = np.argsort(np.max(X_maxplus_orig, axis = 1))[::-1]
    X_maxplus = X_maxplus_orig[row_perm]  # random permutation of rows
    X_maxplus = ma.masked_equal(X_maxplus, missing_value)  # create masked array
    X_maxplus /= X_maxplus.sum()

    X_maxplus_orig = X_maxplus_orig[row_perm]
    X_missing_values = X_missing_values[row_perm]
    
    err, corr_max_rows_decr = run_experiment(X_maxplus_orig, X_missing_values, X_maxplus, rank, max_iter, missing_value, init, init_tmf, repeat)
    
    X_maxplus_orig = copy.deepcopy(X_basic)  # original matrix
    X_missing_values = copy.deepcopy(X_missing)

    columns_perm = np.argsort(np.max(X_maxplus_orig, axis = 0))
    X_maxplus = X_maxplus_orig[:, columns_perm]  # random permutation of rows
    X_maxplus = ma.masked_equal(X_maxplus, missing_value)  # create masked array
    X_maxplus /= X_maxplus.sum()

    X_maxplus_orig = X_maxplus_orig[:, columns_perm]
    X_missing_values = X_missing_values[:, columns_perm]
    
    err, corr_max_columns_incr = run_experiment(X_maxplus_orig, X_missing_values, X_maxplus, rank, max_iter, missing_value, init, init_tmf, repeat)
    
    X_maxplus_orig = copy.deepcopy(X_basic)  # original matrix
    X_missing_values = copy.deepcopy(X_missing)

    columns_perm = np.argsort(np.max(X_maxplus_orig, axis = 0))[::-1]
    X_maxplus = X_maxplus_orig[:, columns_perm]  # random permutation of rows
    X_maxplus = ma.masked_equal(X_maxplus, missing_value)  # create masked array
    X_maxplus /= X_maxplus.sum()

    X_maxplus_orig = X_maxplus_orig[:, columns_perm]
    X_missing_values = X_missing_values[:, columns_perm]
    
    err, corr_max_columns_decr = run_experiment(X_maxplus_orig, X_missing_values, X_maxplus, rank, max_iter, missing_value, init, init_tmf, repeat)
    ####
    data = [corr_spectral, corr_polo, corr_polo_flipped, corr_polo_flipped_rows, corr_mix_1, corr_mix_2, corr_random_rows, corr_random_columns,
       corr_mean_rows_incr, corr_min_rows_incr, corr_max_rows_incr, corr_mean_rows_decr, corr_min_rows_decr, corr_max_rows_decr,
       corr_mean_columns_incr, corr_min_columns_incr, corr_max_columns_incr, corr_mean_columns_decr, corr_min_columns_decr, corr_max_columns_decr]
    data = np.array(data)
    temp_0 = np.median(data, axis=1)
    np.savetxt("perm_techn_1/violin_plots_" + str(seed) + ".csv", temp_0, delimiter=",") 
    temp = np.median(data, axis=1).argsort()
    data = data[temp]
    data = data.tolist()
    labels = ['spectral', 'polo', 'polo flipped', 'polo flipped rows','rows min INCR + columns max INCR', 'rows min DECR + columns max INCR', 'random rows', 'random columns',
         'rows mean INCR', 'rows min INCR', 'rows max INCR', 'rows mean DECR', 'rows min DECR', 'rows max DECR',
         'columns mean INCR', 'columns min INCR', 'columns max INCR', 'columns mean DECR', 'columns min DECR', 'columns max DECR']
    labels = np.array(labels)
    labels = labels[temp]
    labels = labels.tolist()
    ut.violin_plot_all(data, labels, 'Comparison of different permutation techniques', "perm_techn_1/violin_plots_" + str(seed) + ".png")

In [5]:
random_seeds = list(range(35,50)) # seznam parametrov, random seeds from 0 to 25

In [6]:
results = Parallel(n_jobs=len(random_seeds), backend='multiprocessing', timeout=1000000)(
    delayed(generate_wordcloud) (rseed)
    for rseed in random_seeds
)